In [ ]:
%pip install -q transformers datasets loralib sentencepiece bitsandbytes accelerate langchain torch torchvision torchaudio peft numpy protobuf huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!LLAMA_CUBLAS=1
!FORCE_CMAKE=1
!CMAKE_ARGS="-DLLAMA_CUBLAS=ON"
%pip install -q llama-cpp-python --force-reinstall --upgrade --no-cache-dir

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Base ctransformers with no GPU acceleration
# %pip install -q "ctransformers>=0.2.24" --force-reinstall --upgrade --no-cache-dir

# Or with CUDA GPU acceleration
%pip install -q "ctransformers[cuda]>=0.2.24" --force-reinstall --upgrade --no-cache-dir

# Or with ROCm GPU acceleration
# !export CT_HIPBLAS=1
# %pip install -q "ctransformers>=0.2.24" --no-binary ctransformers --force-reinstall --upgrade --no-cache-dir

# Or with Metal GPU acceleration for macOS systems
# !export CT_METAL=1
# %pip install -q "ctransformers>=0.2.24" --no-binary ctransformers --force-reinstall --upgrade --no-cache-dir

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.16.1 requires fsspec[http]<=2023.10.0,>=2023.1.0, but you have fsspec 2024.2.0 which is incompatible.
torch 2.2.0 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.3.4.1 which is incompatible.
torch 2.2.0 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.3.101 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [8]:
from huggingface_hub import hf_hub_download

MODEL_ID = 'davidkim205/komt-Llama-2-13b-hf-ggml'
MODEL_FILENAME = 'ggml-model-q4_0.bin'
# MODEL_FILENAME = 'ggml-model-q8_0.bin'
# MODEL_FILENAME = 'ggml-model-q2_k.bin'

In [9]:
downloaded = hf_hub_download(repo_id=MODEL_ID, filename=MODEL_FILENAME)

In [10]:
import llama_cpp

# llama_model_load_internal: 
#   using CUDA for GPU acceleration llama_model_load_internal: 
#   mem required = 2381.32 MB (+ 1026.00 MB per state) 
#   llama_model_load_internal: allocating batch_size x (512 kB + n_ctx x 128 B) = 480 MB VRAM for the scratch buffer 
#   llama_model_load_internal: offloading 28 repeating layers to GPU 
#   llama_model_load_internal: offloaded 28/35 layers to GPU 
#   llama_model_load_internal: total VRAM used: 3521 MB ...

model = llama_cpp.Llama(
    model_path = f"{downloaded}",
    chat_format="llama-2",
    n_ctx=1024,
    max_tokens=1024,
    n_threads=6,
    n_gpu_layers=30, # Change this value based on your model and your GPU VRAM pool.
    n_batch=256, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    verbose=False,
)

output = model(
    "Q: 인생이란 뭘까요?. A: ",
    max_tokens=1024,
    stop=["Q:", "\n"],
    echo=True
)

print( output['choices'][0]['text'].replace('▁',' ') )

print(model.create_chat_completion(
    messages=[{
        "role": "user",
        "content": "what is the meaning of life?"
    }]
))

gguf_init_from_file: invalid magic characters 'tjgg'
llama_model_load: error loading model: llama_model_loader: failed to load model from /home/tinywind/.cache/huggingface/hub/models--davidkim205--komt-Llama-2-13b-hf-ggml/snapshots/ffa5028d28327db5b355b8b2282a298fb6b355af/ggml-model-q4_0.bin

llama_load_model_from_file: failed to load model


AssertionError: 

In [11]:
from langchain.llms import LlamaCpp, HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from langchain.callbacks.manager import CallbackManager
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler # 출력을 스트리밍하는 데 사용
import torch

local_llm = LlamaCpp(
    model_path = f"{downloaded}",
    input={
        "temperature": 0.75,
        "max_length": 2000,
        "top_p": 1
    },
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

llm_chain = LLMChain(
    llm=local_llm,
    prompt=PromptTemplate(input_variables=["question"], template="### 질문:\n{intruction}\n\n답변:"),
)

conversation = ConversationChain(
    llm=local_llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=4)
)

conversation.prompt.template = '''대화기록:
{history}
질문: {input}
답변:'''

def ask(question):
    print(llm_chain.invoke(question))

def chat(input):
    print(conversation.predict(input=input))

Exception ignored in: <function _LlamaContext.__del__ at 0x7f8f246dc820>
Traceback (most recent call last):
  File "/home/tinywind/anaconda3/envs/alpaca-with-langchain/lib/python3.10/site-packages/llama_cpp/_internals.py", line 259, in __del__
    if self.ctx is not None and self._llama_free is not None:
AttributeError: '_LlamaContext' object has no attribute 'ctx'
/home/tinywind/anaconda3/envs/alpaca-with-langchain/lib/python3.10/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! input is not default parameter.
                input was transferred to model_kwargs.
                Please confirm that input is what you intended.
  warnings.warn(
gguf_init_from_file: invalid magic characters 'tjgg'
llama_model_load: error loading model: llama_model_loader: failed to load model from /home/tinywind/.cache/huggingface/hub/models--davidkim205--komt-Llama-2-13b-hf-ggml/snapshots/ffa5028d28327db5b355b8b2282a298fb6b355af/ggml-model-q4_0.bin

llama_load_model_from_file: fail

ValidationError: 1 validation error for LlamaCpp
__root__
  Could not load Llama model from path: /home/tinywind/.cache/huggingface/hub/models--davidkim205--komt-Llama-2-13b-hf-ggml/snapshots/ffa5028d28327db5b355b8b2282a298fb6b355af/ggml-model-q4_0.bin. Received error  (type=value_error)

In [12]:
from langchain.llms import CTransformers

llm = CTransformers(model=MODEL_ID, model_path = f"{downloaded}", model_type='llama')
print(llm('LLM Jindo is going to'))

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

ggml-model-q2_k.bin:   0%|          | 0.00/5.51G [00:00<?, ?B/s]

In [ ]:
from langchain.llms import CTransformers
from langchain import PromptTemplate, LLMChain
from langchain.chains import ConversationChain
from langchain.callbacks.manager import CallbackManager
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler # 출력을 스트리밍하는 데 사용
import torch

local_llm = CTransformers(
    model=MODEL_ID,
    model_path = f"{downloaded}",
    model_type='llama',
    input={
        "temperature": 0.9,
        "max_length": 500,
        "top_p": 1
    },
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

llm_chain = LLMChain(
    llm=local_llm,
    prompt=PromptTemplate(input_variables=["question"], template="### 질문:\n{intruction}\n\n답변:"),
)

conversation = ConversationChain(
    llm=local_llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=4)
)

conversation.prompt.template = '''대화기록:
{history}
질문: {input}
답변:'''

def ask(question):
    print(llm_chain.invoke(question))

def chat(input):
    print(conversation.predict(input=input))

In [ ]:
ask('What is the capital of England?')

In [ ]:
ask('내 이름이 뭘까?')

In [ ]:
chat('나의 이름은 전재형이야')

In [ ]:
chat('니 이름은 뭐니?')

In [ ]:
chat('내 이름은 뭘까??')